# 1D solute transport with advection and dispersion

The Jupyter Notebook computes 1D solute transport with advection and dispersion (Ogata and Banks 1961). Parameters to modify:

- $\alpha$ (Dispersivity) and
- $n$ (Porosity).

In [ ]:
# %matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from scipy import special
import numpy as np
from ipywidgets import *
#from ipynb.fs.full.BHYWI04_functions import create_multipleChoice_widget
#from ipynb.fs.full.BHYWI04_functions import create_multipleChoice_widget_PDF

#FUNCTIONS FOR COMPUTATION; ADS = ADVECTION, DISPERSION AND SORPTION - EVENTUALLY SET RETARDATION TO 1 FOR NO SORPTION

def IC(PE,r_time):
    IC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (IC1>0):
        IC2 = 1-(1-special.erfc(abs(IC1)))
    else:
        IC2 = 1+(1-special.erfc(abs(IC1)))
    IC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    if (IC3>0):
        IC4 = 1-(1-special.erfc(abs(IC3)))
    else:
        IC4 = 1+(1-special.erfc(abs(IC3)))
    if IC4 == 0:
        IC5 = IC2
    else:
        IC5 = IC2+np.exp(PE)*IC4
    IC  = 1-0.5*IC5
    return IC

def BC(PE,r_time, r_dur):
    # BCx positive pulse
    BC1 = np.sqrt(0.25*PE/r_time)*(1-r_time)
    if (BC1>0):
        BC2 = 1-(1-special.erfc(abs(BC1)))
    else:
        BC2 = 1+(1-special.erfc(abs(BC1)))
    BC3 = np.sqrt(0.25*PE/r_time)*(1+r_time)
    BC4 = special.erfc(BC3)
    if BC4 == 0:
        BC5 = BC2
    else:
        BC5 = BC2 + np.exp(PE) * BC4
    
    # BCCx negative pulse
    if r_time > r_dur:
        BCC1 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 - (r_time - r_dur))
        if BCC1 > 0:
            BCC2 = 1 - (1 - special.erfc(abs(BCC1)))
        else:
            BCC2 = 1 + (1 - special.erfc(abs(BCC1)))  
        BCC3 = np.sqrt(0.25 * PE / (r_time - r_dur)) * (1 + (r_time - r_dur))
        BCC4 = special.erfc(BCC3)
        if BCC4 == 0:
            BCC5 = BCC2
        else:
            BCC5 = BCC2 + np.exp(PE) * BCC4      
    if r_time <= r_dur:
        BC = 0.5 * BC5
    else:
        BC = 0.5 * (BC5 - BCC5)
    return BC

def transport(l,t1,c0,m,Q,n,a,plot_A, plot_AD, plot_DATA):
    # l : länge der säule, float
    # t1 : versuchsende, float
    # ci : initiale konzentration, float
    # c0 : eingabekonzentration, float
    # cp : max. conc plot
    # m : eingabemasse, float
    # Q : durchfluss, float
    # n : porosität, float
    # a : dispersivität (a > 0), float
    
    # Data for plotting
    t0 = 1       #Startzeit
    dt = 2      #Zeitdiskretisierung
    r  = 2    #Radius der Säule
    ci = 0
    cp = 2 * c0
    
    #Berechnung Zwischenergebnisse
    A =     np.pi*r**2
    q =     Q/A
    v =     q/n
    D =     a*v
    PE =    l/a
    dur =   m/Q/(c0-ci)
    tPV =   l/v
    r_dur = dur/tPV
    r_dt =  dt/tPV

    #Festlegung Zeitbereich
    t = np.arange(t0, t1, dt)

    #Berechnung Konzentration - Klammerterme
    #Set fraction of distance
    r_time = []
    time   = []
    conc   = []
    conca  = []
    
    #compute concentration  
    for t in range(t0, t1, dt):      
        r_time = t/tPV
        # ADVECTION-DISPERSION
        c = ci*IC(PE,r_time)+c0*BC(PE,r_time, r_dur)
        conc.append(c)
        # ADVECTION ONLY
        if r_time < 1:
            ca = 0
        elif r_time > 1+r_dur:
            ca = 0
        else:
            ca = c0
        conca.append(ca)
        time.append(t)
        
    # measurements
    t_obs = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]
    c_obs = [1e-3, 5e-2, 8.5e-2, 9.7e-2, 9.9e-2, 9e-2, 5e-2, 1.5e-2, 2e-3, 5e-4]
   
    #PLOT FIGURE
    fig = plt.figure(figsize=(9,6))
    ax = fig.add_subplot(1, 1, 1)
    ax.set_title('Advection-dispersion transport', fontsize=14)
    ax.set_xlabel ('Time', fontsize=14)
    ax.set_ylabel ('Concentration', fontsize=14)
    #ax.set(xlabel='Zeit', ylabel='Konzentration', title='A-D transport', fontsize=14)
        
# PLOT HERE
    if plot_A == 1:
        ax.plot(time,conca, 'fuchsia', linewidth=2, label="(only) Advektion - computed")
    if plot_AD == 1:
        ax.plot(time,conc, 'navy', linewidth=2, label="Advektion-Dispersion - computed")
    if plot_DATA == 1:
        ax.plot(t_obs, c_obs, 'ro', label="Measured")
    #ax.scatter(t_obs, c_obs, marker="x", c="red", zorder=10)
    plt.ylim(0, cp)
    plt.xlim(0,t1)
    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)
    if not plot_A !=1 and plot_AD != 1 and plot_DATA != 1:
        plt.legend(frameon=False, loc='upper right', fontsize=14)
    #ax.grid()

    plt.show()
    
#THE FOLLOWING ALLOWS TO DEFINE THE RANGE OF SLIDERS DEPENDING ON OTHER SLIDER SETTINGS (HERE DISP AS FUNCTION OF LENGTH)
interact(transport,
         #DEFINE THE RANGE SUCH THAT NOT TOO MANY COMPUTATIONS ARE NECESSARY WHEN MOVING THE SLIDER!
         l  = FloatText(value=15,min=1, max=100,step=1,description='Lenght', readout_format='.2f' ),
         t1 = widgets.IntText(value=1800, min = 60, max = 86400, step = 60, description = 'Time max'),
         c0 = widgets.FloatText(value=0.1,min=0.01, max=5,step=0.01,description='Mass input conc.', readout_format='.2f' ),
         m  = widgets.FloatText(value=10,min=0, max=1000,step=1,description='Mass input', readout_format='.2f' ),
         Q  = widgets.FloatText(value=0.2, min=0.00001, max=0.001, step=0.0001, description='Discharge Q',readout_format='.5f'),
         n  = widgets.FloatSlider(value=0.2,min=0.02, max=0.6, step=0.001, description='n',readout_format='.3f'),       
         a  = FloatSlider(value=0.01,min=0.001, max=10,step=0.002,description='alpha', readout_format='.3f' ),
         plot_A    = widgets.Checkbox(value=True, description='Plot only advection ',disabled=False),
         plot_AD   = widgets.Checkbox(value=False, description='Plot advection-dispersion ',disabled=False),
         plot_DATA = widgets.Checkbox(value=False, description='Plot measured data ',disabled=False),
        )

<hr>
Jupyter Notebook developed by Thomas Reimann & Max Rudolph, based on a XLS-Worksheet of Prof. Rudolf Liedl.

Last change 2024 08 21 by Thomas.Reimann@tu-dresden.de

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>